# Using Java Info Class 

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.Info;
import com.aerospike.client.cluster.Node;
import java.util.Map;
System.out.println("Client modules imported.");

Client modules imported.


In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

# Insert 10 records

In [11]:
%sh aql -f "../aqlScripts/insert.aql" -h "127.0.0.1"

# Connect to Remote Cluster

In [12]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to remote cluster.");


/*
private void GetNamespaceConfig(Node node, String namespace) {
  //String filter = "namespace/" + namespace;
  String filter = "namespace/" + namespace;
  // Issue the info request for the namespace
  String tokens = Info.request(null, node, filter);
  int masterObjects = GetTokenMasterObjects(tokens);
}
*/

private int GetTokenMasterObjects(String tokens) {
  String[] values = tokens.split(";");
  String MASTER_OBJECT_COUNT_TAG = "master_objects=";
  int MASTER_OBJECT_COUNT_TAG_LENGTH = MASTER_OBJECT_COUNT_TAG.length();
  for (String value : values) {
   if (value.startsWith(MASTER_OBJECT_COUNT_TAG)) {
    return Integer.parseInt(value.substring(MASTER_OBJECT_COUNT_TAG_LENGTH));
   }
  }
  return 0;
}

private int GetSetObjects(String tokens) {
  String[] values = tokens.split(":");
  String SET_OBJECT_COUNT_TAG = "objects=";
  int SET_OBJECT_COUNT_TAG_LENGTH = SET_OBJECT_COUNT_TAG.length();
  for (String value : values) {
    if (value.startsWith(SET_OBJECT_COUNT_TAG)) {
      return Integer.parseInt(value.substring(SET_OBJECT_COUNT_TAG_LENGTH));
    }
  }
  return 0;
}

private int GetMasterObjectCount(Node node, String namespace) {
  String filter = "namespace/" + namespace;
  String tokens = Info.request(null, node, filter);
  //return the master-objects value
  return(GetTokenMasterObjects(tokens));        
}

private int GetSetObjectCount(Node node, String namespace, String set) {
  String filter = "sets/" + namespace + "/" + set;
  String tokens = Info.request(null, node, filter);
  //return the master-objects value
  return(GetSetObjects(tokens));        
}

private String GetClusterKey(Node node, String namespace, int size) {
  String filter = "cluster-stable:size=" + size+"; namespace="+namespace;
  String key = Info.request(null, node, filter);
  //return the master-objects value
  return(key);        
}

private String SetServiceConfigInt(Node node, String param, int val) {
  String filter = "set-config:context=service;" + param +"="+val;
  String key = Info.request(null, node, filter);
  //return the master-objects value
  return(key);        
}

//Run Info 
  
int masterObjectCount = 0;
int setObjectCount = 0;
int clusterSize = 1;
Node nodes[] = client.getNodes();
String ns = "test";
String set = "test2set";

// For each node in the cluster
String clusterKey=null;
String setResponse=null;
for (Node node : nodes) {
  System.out.println("node: " + node + GetClusterKey(node, ns, clusterSize));
  String thisNodeClusterKey = GetClusterKey(node, ns, clusterSize);
  if(clusterKey==null){clusterKey = thisNodeClusterKey;} 
  if(clusterKey.equals(thisNodeClusterKey)){
    // Get it's master-objects value and add to the total
    masterObjectCount += GetMasterObjectCount(node, ns);
    setObjectCount += GetSetObjectCount( node, ns, set);
    setResponse = SetServiceConfigInt(node, "query-threads-limit", 64); 
    System.out.println("node:"+node+":"+setResponse);  //Add check if all nodes responed OK
      
  }
  else {System.out.println("Cluster unstable"+clusterKey+": "+thisNodeClusterKey);}
}
//Check cluster-key did not change
for (Node node : nodes) {
  //System.out.println("node: " + node + GetClusterKey(node, ns, clusterSize));
  String thisNodeClusterKey = GetClusterKey(node, ns, clusterSize);
  if(!(clusterKey.equals(thisNodeClusterKey))){
      System.out.println("Cluster unstable, repeat command."+clusterKey+": "+thisNodeClusterKey);
      //Add code to repeat the command if cluster became unstable during the command execution
  }
}

System.out.println("master objects: " + masterObjectCount);
System.out.println("set objects: " + setObjectCount);

Initialized the client and connected to remote cluster.
node: A1 127.0.0.1 3000F085CB5C48CA
node:A1 127.0.0.1 3000:ok
master objects: 10
set objects: 0


In [13]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to remote cluster.");

private String GetClusterKey(Node node, String namespace, int size) {
  String filter = "cluster-stable:size=" + size+"; namespace="+namespace;
  String key = Info.request(null, node, filter);
  //return the master-objects value
  return(key);        
}

private String SetServiceConfigInt(Node node, String param, int val) {
  String filter = "set-config:context=service;" + param +"="+val;
  String key = Info.request(null, node, filter);
  //return the master-objects value
  return(key);        
}

//Run Info 
  
Node nodes[] = client.getNodes();
String ns = "test";

// For each node in the cluster
String clusterKey=null;
String setResponse=null;
int clusterSize = 1;
for (Node node : nodes) {
  System.out.println("node: " + node + GetClusterKey(node, ns, clusterSize));
  String thisNodeClusterKey = GetClusterKey(node, ns, clusterSize);
  if(clusterKey==null){clusterKey = thisNodeClusterKey;} 
  if(clusterKey.equals(thisNodeClusterKey)){
    setResponse = SetServiceConfigInt(node, "query-threads-limit", 128); 
    System.out.println("node:"+node+":"+setResponse);  //Add check if all nodes responed OK
      
  }
  else {System.out.println("Cluster unstable"+clusterKey+": "+thisNodeClusterKey);}
}
//Check cluster-key did not change
for (Node node : nodes) {
  //System.out.println("node: " + node + GetClusterKey(node, ns, clusterSize));
  String thisNodeClusterKey = GetClusterKey(node, ns, clusterSize);
  if(!(clusterKey.equals(thisNodeClusterKey))){
      System.out.println("Cluster unstable, repeat command."+clusterKey+": "+thisNodeClusterKey);
      //Add code to repeat the command if cluster became unstable during the command execution
  }
}


Initialized the client and connected to remote cluster.
node: A1 127.0.0.1 3000F085CB5C48CA
node:A1 127.0.0.1 3000:ok


# Cleanup

In [14]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"